## 一、环境准备


In [2]:
pip install scipy

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


## 二、复习分类算法的简单实现

In [4]:
from sklearn import datasets
iris = datasets.load_iris()

In [5]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [6]:
y_pred = gnb.fit(iris.data, iris.target).predict(iris.data)

In [7]:
print("Number of mislabeled points out of a total %d points : %d" % (iris.data.shape[0],(iris.target != y_pred).sum()))

Number of mislabeled points out of a total 150 points : 6


## 三、人群的收入预测 现在我们获得了一个数据集adult.data.txt

## STEP1：加载数据并查看数据，我们希望依据不同的类别收入加载相同数量的数据，因为如果不同的类别的数据体量相差很大会造成分类器向大数据体量一方倾斜。

In [10]:
import numpy as np
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB 

###STEP1###
# 加载数据
input_file = 'D:/jupyter file/date/adult.data.txt'
X = []
y = []
num_lessthan50k = 0
num_morethan50k = 0
num_threshold = 30000
with open(input_file, 'r') as f:
    for line in f.readlines():
        if '?' in line:
            continue
        # 逗号后有个空格
        data = line[:-1].split(', ')
        if data[-1] == '<=50K' and num_lessthan50k < num_threshold:
            X.append(data)
            num_lessthan50k = num_lessthan50k + 1
# 依据target的不同，各选取相同大小的数据，即选取'<=50K'与'>50K'数量相等的数据
        elif data[-1] == '>50K' and num_morethan50k < num_threshold:
            X.append(data)
            num_morethan50k = num_morethan50k + 1
        if num_lessthan50k >= num_threshold and num_morethan50k >= num_threshold:
            break
X = np.array(X)

## STEP2：转换数据的属性编码，因为原数据中的属性是包含英文字符，无法进行数学运算，请将其转换为数值型数据。

In [11]:
label_encoder = [] 
X_encoded = np.empty(X.shape)
for i,item in enumerate(X[0]):
    if item.isdigit(): 
        X_encoded[:, i] = X[:, i]
    else:
        le=preprocessing.LabelEncoder()
        label_encoder.append(le)
        # 转换数据的属性，已知原数据中含有英文字符，请将英文转换为数值型，使用preprocessing.LabelEncoder()函数
        X_encoded[:, i] = label_encoder[-1].fit_transform(X[:, i])
X = X_encoded[:, :-1].astype(int)
y = X_encoded[:, -1].astype(int)

## STEP3：创建一个分类器，将STEP1、STEP2处理好的数据进行训练，并使用交叉验证评测分类器的效果。

In [13]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)
classifier_gaussiannb = GaussianNB()
classifier_gaussiannb.fit(X_train, y_train)
y_test_pred = classifier_gaussiannb.predict(X_test)
# 计算F1得分
f1 = cross_val_score(classifier_gaussiannb,
        X, y, scoring='f1_weighted', cv=5)
print ("F1 score: " + str(round(100*f1.mean(), 2)) + "%")

F1 score: 75.9%


## STEP4：对新个体采用同样的编码处理，预测其收入类别。

In [14]:
# 创建个例，将其进行同样编码处理
input_data = ['39', 'State-gov', '77516', 'Bachelors', '13', 'Never-married', 'Adm-clerical', 'Not-in-family', 'White', 'Male', '2174', '0', '40', 'United-States'] 
count = 0
input_data_encoded = [-1] * len(input_data)
for i,item in enumerate(input_data):
    if item.isdigit():
        input_data_encoded[i] = int(input_data[i])
    else:
        input_data_encoded[i] = int(label_encoder[count].transform([input_data[i]]))
        count = count + 1 
input_data_encoded = np.array(input_data_encoded)
#将个体进行预测分类，并输出结果
output_class = classifier_gaussiannb.predict(input_data_encoded.reshape(1,-1))
print (label_encoder[-1].inverse_transform(output_class)[0])

<=50K
